In [29]:
import bs4 as bs
import os
import pickle
import requests
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn
import datetime as dt
from matplotlib import style
import mplfinance as mpf
import matplotlib.dates as mdates
import pandas_datareader.data as web
import yfinance as yf
import seaborn as sns
from collections import Counter
from sklearn import svm, neighbors
from sklearn.model_selection import train_test_split
from sklearn.ensemble import VotingClassifier, RandomForestClassifier


style.use('ggplot')

In [6]:
hm_days = 7

def process_data_for_labels(ticker):
    df = pd.read_csv('join_closes.csv', index_col=0)
    tickers = df.columns.values.tolist()
    df.fillna(0, inplace=True)
    
    for i in range(1, hm_days + 1):
        df['{}_{}d'.format(ticker, i)] = (df[ticker].shift(-i) - df[ticker]) / df[ticker]
    
    df.fillna(0, inplace=True)
    
    return tickers, df

In [51]:
def buy_sell_hold(*args):
    cols = [c for c in args] 
    requirement = 0.02
    for col in cols:
        if col > requirement:
            return 1
        elif col < requirement:
            return -1
    return 0

In [52]:
def extract_feature_sets(ticker):
    tickers, df = process_data_for_labels(ticker)
    df['{}_target'.format(ticker)] = list(map( buy_sell_hold,
                                               df['{}_2d'.format(ticker)],
                                               df['{}_3d'.format(ticker)],
                                               df['{}_4d'.format(ticker)],
                                               df['{}_5d'.format(ticker)],
                                               df['{}_6d'.format(ticker)],
                                               df['{}_7d'.format(ticker)]
                                               ))
    vals = df['{}_target'.format(ticker)].values.tolist()
    str_vals = [str(i) for i in vals]
    print('Data Spread:', Counter(str_vals))
    df.fillna(0, inplace=True)
    
    df = df.replace([np.inf, -np.inf], np.nan)
    df.dropna(inplace=True)
    
    df_vals = df[[ticker for ticker in tickers]].pct_change()
    df_vals = df_vals.replace([np.inf, -np.inf], 0)
    df_vals.fillna(0, inplace=True)
    
    
    X = df_vals.values
    y = df['{}_target'.format(ticker)].values
    
    return X, y, df

In [54]:
def do_ml(ticker):
    X, y, df = extract_feature_sets(ticker)
     
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    
    #clf = neighbors.KNeighborsClassifier()
    
    clf = VotingClassifier([('lsvc', svm.LinearSVC()), ('knn', neighbors.KNeighborsClassifier()), ('forest', RandomForestClassifier())])
    
    clf.fit(X_train, y_train)
    confidence = clf.score(X_test, y_test)
    predictions = clf.predict(X_test)
    
    print('Predicted Spread:', Counter(predictions))
    
    return confidence

do_ml('TSLA\n')

Data Spread: Counter({'-1': 676, '1': 350})
Predicted Spread: Counter({-1: 192, 1: 14})


0.6456310679611651